In [10]:
import os
import re

# os.environ["OPENAI_API_KEY"] = "sk-your-api-key-here"

In [14]:
from importlib import reload

import httpx
from openai import OpenAI

import src.agents as agents

In [6]:
client = OpenAI()
client.api_key = os.getenv("OPENAI_API_KEY")

## ReAct prompt

In [34]:
react_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:
calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point
syntax if necessary

wikipedia:
e.g. wikipedia: Django
Returns a summary from searching Wikipedia

Example session:
Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:
Observation: France is a country. The capital is Paris.

You then output:
Answer: The capital of France is Paris
""".strip()

actions_re = re.compile("^Action: (\w+): (.*)")

## Actions

In [28]:
def wikipedia(q):
    response = httpx.get(
        "https://en.wikipedia.org/w/api.php",
        params={"action": "query", "list": "search", "srsearch": q, "format": "json"},
    )
    return response.json()["query"]["search"][0]["snippet"]


def calculate(query: str):
    return eval(query)


known_actions = {"wikipedia": wikipedia, "calculate": calculate}

In [35]:
reload(agents)


def query_agent(query: str, max_turns: int = 10):
    i = 0
    bot = agents.BaseAgent(client=client, system_prompt=react_prompt)
    next_prompt = query
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [
            actions_re.match(a) for a in result.split("\n") if actions_re.match(a)
        ]
        if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception(f"Unknown action: {action}: {action_input}")
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = f"Observation: {observation}"
        else:
            return result


print(query_agent("What does England share borders with?"))

Thought: England is part of the United Kingdom, and I should look up its geographical borders to learn what countries it shares borders with. Specifically, I will check if it shares borders with any countries or other regions. 

Action: wikipedia: England borders

PAUSE
 -- running wikipedia England borders
Observation: Anglo-Scottish <span class="searchmatch">border</span> (Scottish Gaelic: Crìochan Anglo-Albannach) is an internal <span class="searchmatch">border</span> of the United Kingdom separating Scotland and <span class="searchmatch">England</span> which runs for
Answer: England shares a border with Scotland.
Answer: England shares a border with Scotland.


In [36]:
print(query_agent("What is the age difference between the Queen and Prince Charles?"))

Thought: I need to find out the birth dates of both the Queen and Prince Charles to calculate the age difference between them. I should look up their birth dates on Wikipedia.  
Action: wikipedia: Queen Elizabeth II birth date  
PAUSE  
 -- running wikipedia Queen Elizabeth II birth date  
Observation: <span class="searchmatch">Elizabeth</span> <span class="searchmatch">II</span> (<span class="searchmatch">Elizabeth</span> Alexandra Mary; 21 April 1926 – 8 September 2022) was <span class="searchmatch">Queen</span> of the United Kingdom and other Commonwealth realms from 6 February
Thought: I have the birth date of Queen Elizabeth II, which is April 21, 1926. Now I need to find out the birth date of Prince Charles. I'll look it up on Wikipedia.  
Action: wikipedia: Prince Charles birth date  
PAUSE  
 -- running wikipedia Prince Charles birth date  
Observation: Isles, and <span class="searchmatch">Prince</span> and Great Steward of Scotland. The following year, <span class="searchmatch

In [95]:
import src.agents as agents
import src.llm.openai as openai_llm

reload(agents)

prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:
calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point
syntax if necessary

wikipedia:
e.g. wikipedia: Django
Returns a summary from searching Wikipedia

Example session:
Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:
Observation: France is a country. The capital is Paris.

You then output:
Answer: The capital of France is Paris
""".strip()

actions_re = re.compile("^Action: (\w+): (.*)")


def wikipedia(q):
    wiki_wiki = wikipediaapi.Wikipedia('MyProjectName', 'en')
    summary = wiki_wiki.page(q).summary

    return summary


def calculate(query: str):
    return eval(query)


wikipedia_tool = agents.Tool(
    name="wikipedia", description="Search Wikipedia", function=wikipedia
)

calculate_tool = agents.Tool(
    name="calculate", description="Run a calculation", function=calculate
)

llm = openai_llm.OpenAI(model="gpt-4o-mini")

my_agent = agents.Agent(llm=llm, max_turns=10, system_prompt=prompt)

my_agent.add_tool(wikipedia_tool)
my_agent.add_tool(calculate_tool)

res = my_agent.run("How tall is Trump Towwer?")

Thought: I need to find the height of Trump Tower, which is a well-known building. A quick look at Wikipedia should provide accurate information about its height.  
Action: wikipedia: Trump Tower  
PAUSE
----------
* Tool called:
	Name: wikipedia
	Input: Trump Tower  
----------
Observation: Trump Tower is a 58-story, 664-foot-tall (202 m) mixed-use condominium skyscraper at 721–725 Fifth Avenue in the Midtown Manhattan neighborhood of New York City, between East 56th and 57th Streets. The building contains the headquarters for the Trump Organization, as well as the penthouse residence of its developer, the businessman and former U.S. president Donald Trump. Several members of the Trump family also live, or have lived, in the building. The tower stands on a plot where the flagship store of the department-store chain Bonwit Teller was formerly located.
Der Scutt of Swanke Hayden Connell Architects designed Trump Tower, and Trump and the Equitable Life Assurance Company (now the AXA Equi

In [80]:
import wikipediaapi

def wikipedia(q):
    wiki_wiki = wikipediaapi.Wikipedia('MyProjectName', 'en')
    summary = wiki_wiki.page(q).summary

    return summary

In [89]:
wikipedia("Barack Obama")

"Barack Hussein Obama II (born August 4, 1961) is an American politician who served as the 44th president of the United States from 2009 to 2017. As a member of the Democratic Party, he was the first African-American  president in U.S. history. Obama previously served as a U.S. senator representing Illinois from 2005 to 2008 and as an Illinois state senator from 1997 to 2004. \nObama was born in Honolulu, Hawaii. He graduated from Columbia University in 1983 with a Bachelor of Arts degree in political science and later worked as a community organizer in Chicago. In 1988, Obama would enroll in Harvard Law School, where he became the first black president of the Harvard Law Review. He became a civil rights attorney and an academic, teaching constitutional law at the University of Chicago Law School from 1992 to 2004. He also went into elective politics; Obama represented the 13th district in the Illinois Senate from 1997 until 2004, when he successfully ran for the U.S. Senate. In the 20